In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import sys
import pyodbc
import time

import myfunctions as myf
import qadconnect34plus as q

%pylab inline

sys.path.append('S:/Stoxx/Product Development and Research/Python')

dloc ='S:/Stoxx/Product Development and Research/Projects/2393 MUTB Quality Overlay on China A BRIC Russia Asia ex Japan/'
histloc = dloc + '09 History/'
calcloc = histloc + '01_calc_steps/'

Populating the interactive namespace from numpy and matplotlib


In [18]:
universe = 'sxcna9p'

df = pd.read_excel(histloc + '/universe/' + universe + '.xlsx')
df['ICB'] = df['ICB'].map(lambda x: str(x))

dfind = pd.read_excel(histloc + '/universe/industry_codes.xlsx',sheetname='full')
df = pd.merge(df, dfind[[0,1,2]], how='left', left_on='ICB', right_on='old_subs')

del df['old_subs'] 
df.to_csv(calcloc + '01_china900.csv', sep=';', index=False)

In [37]:
dfsedcor = pd.read_excel(histloc + 'universe/sedol_correction.xlsx')

dfsedcor

for index, row in dfsedcor.iterrows():
    print(row.sedol)
    df.loc[df[df.sedol==row.sedol].index, 'sedol'] = row.sedol_correct
df['SEDOL'] = df['sedol']
del df['sedol']
df

B1KKCM
B01XL1
B03NNP
B1N93W
B56353
651973
280192


,Next_Trading_Day,Index_Name,Index_Currency,Internal_Number,ISIN,SEDOL,RIC,Instrument_Name,Country,Currency,Exchange,ICB,Shares,Free_Float,Close_unadjusted_local,Close_adjusted_local,Mcap_Units_local,Mcap_Units_Index_Currency
0,2010-12-20,STXCNA900 EUR P,EUR,NaN,CNE1000007Q1,B28SLD,601857.SS,NaN,CN,CNY,NaN,537,NaN,NaN,NaN,NaN,NaN,2.138733e+11
1,2010-12-20,STXCNA900 EUR P,EUR,NaN,CNE000000MV7,640397,600759.SS,NaN,CN,CNY,NaN,4577,NaN,NaN,NaN,NaN,NaN,7.834850e+08
2,2010-12-20,STXCNA900 EUR P,EUR,NaN,CNE000001DT8,660389,600343.SS,NaN,CN,CNY,NaN,2757,NaN,NaN,NaN,NaN,NaN,7.815994e+08
3,2010-12-20,STXCNA900 EUR P,EUR,NaN,CNE000001FQ9,667619,600487.SS,NaN,CN,CNY,NaN,9578,NaN,NaN,NaN,NaN,NaN,7.801111e+08
4,2010-12-20,STXCNA900 EUR P,EUR,NaN,CNE000001GS3,674381,600438.SS,NaN,CN,CNY,NaN,3573,NaN,NaN,NaN,NaN,NaN,7.800587e+08
5,2010-12-20,STXCNA900 EUR P,EUR,NaN,CNE000000GB1,641987,600831.SS,NaN,CN,CNY,NaN,5553,NaN,NaN,NaN,NaN,NaN,7.776894e+08
6,2010-12-20,STXCNA900 EUR P,EUR,NaN,CNE000000D24,620426,000426.SZ,NaN,CN,CNY,NaN,7535,NaN,NaN,NaN,NaN,NaN,7.724880e+08
7,2010-12-20,STXCNA900 EUR P,EUR,NaN,CNE000000QD6,611343,600067.SS,NaN,CN,CNY,NaN,2733,NaN,NaN,NaN,NaN,NaN,7.706305e+08
8,2010-12-20,STXCNA900 EUR P,EUR,NaN,CNE000001D72,657763,600563.SS,NaN,CN,CNY,NaN,2733,NaN,NaN,NaN,NaN,NaN,7.702240e+08
9,2010-12-20,STXCNA900 EUR P,EUR,NaN,CNE000000TF5,609308,600117.SS,NaN,CN,CNY,NaN,1757,NaN,NaN,NaN,NaN,NaN,7.698450e+08


In [40]:
#clean sedols
ix = 'sxcna9p' #'ciasxj' #'cibric' # 'sxcna9p'
df = pd.read_excel(histloc + 'universe/' + ix + '.xlsx')
dfsedcor = pd.read_excel(histloc + 'universe/' + ix + '.xlsx')

sedol_list = []
dfids = df[['ISIN']].drop_duplicates().reset_index(drop=True)
print(len(dfids))
for index, row in dfids.iterrows():
    #print(row.ISIN)
    sedol_list.append(q.get_sedol(row.ISIN))

    if not (index+1) % 300:
        print(index+1)

dfids['sedol'] = sedol_list
df = pd.merge(df, dfids, how='left', on=['ISIN']).drop_duplicates()

dfsedcor = pd.read_excel(histloc + 'universe/sedol_correction.xlsx')

for index, row in dfsedcor.iterrows():
    #print(row.sedol)
    df.loc[df[df.sedol==row.sedol].index, 'sedol'] = row.sedol_correct
df['SEDOL'] = df['sedol']

del df['sedol']

#df.to_excel(histloc + 'universe/' + ix + '_sedols.xlsx', index=False)
df.to_excel(histloc + 'universe/' + ix + '_.xlsx', index=False)
#q.get_prod_wswspit(3995, sedstr, dat, 'N', 'wspit', -12)
print('done')

1543
300
600
900
1200
1500
done


In [22]:
#download to check which are the best sedols

dfx = df[['Next_Trading_Day','ISIN','SEDOL','sedol']]

dfx['sedol_'] = dfx['SEDOL'].map(lambda x: str(x)[:6])
del dfx['SEDOL']

def get_sqlstr(sedlst):
    sedstr = ''
    for s in sedlst:
        sedstr = sedstr + s[:6] + ","
    return sedstr[:-1].replace("'","")

df2 = pd.DataFrame()

print(len(dfx))
ct=0
print(len(dfx['Next_Trading_Day'].drop_duplicates()))
for d in sorted(dfx['Next_Trading_Day'].drop_duplicates().reset_index(drop=True)):

    sedlst =list(dfx[dfx.Next_Trading_Day==d]['sedol'])
    sedstr = get_sqlstr(sedlst)
    
    sedlst_ =list(dfx[dfx.Next_Trading_Day==d]['sedol_'])
    sedstr_ = get_sqlstr(sedlst_)
    
    df1 = dfx[dfx.Next_Trading_Day==d].reset_index(drop=True)
    
    dat = str(d)[:10]
    
    dffund = q.get_prod_wswspit(3995, sedstr, dat, 'Y', 'wspit', -12)[['sedol','value']]
    dffund = dffund.rename(columns={'sedol':'sedol','value':str(3995)})
    
    dffund_ = q.get_prod_wswspit(3995, sedstr_, dat, 'Y', 'wspit', -12)[['sedol','value']]
    dffund_ = dffund_.rename(columns={'sedol':'sedol_','value':str(3995) + '_'})
         
    df1 = pd.merge(df1, dffund, on='sedol', how='left')
    df1 = pd.merge(df1, dffund_, on='sedol_', how='left')
            
    if len(df2)==0:
        df2 = df1
    else:
        df2 = pd.concat([df2, df1], axis=0)
        
    print(ct)
    ct=ct+1
    
print('done')
df2.to_excel(histloc + 'universe/' + ix + '_sedols_wsdata.xlsx', index=False)

C:\Users\iv822\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


22500
25
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
done


In [23]:
q.get_infocode('CNE000000404')

159723

In [ ]:
#get all infocodes
def do_step1():
    
    df = pd.read_csv(calcloc + '01_china900.csv', sep=';')
    df = df.iloc[:,:]

    sedol_list = []
    dfids = df[['ISIN']].drop_duplicates().reset_index(drop=True)
    print(len(dfids))
    for index, row in dfids.iterrows():
        sedol_list.append(q.get_sedol(row.ISIN))

        if not (index+1) % 300:
            print(index+1)

    dfids['sedol'] = sedol_list
    df = pd.merge(df, dfids, how='left', on=['ISIN']).drop_duplicates()

    #df.read_excel(histloc + 'universe/sedol_correction.xlsx') #use sedol correction file
    
    df.to_csv(calcloc + '02_china900_codes.csv', sep=';', index=False)

    print('done step1')

In [31]:
#get all infocodes
def do_step2():
    
    df = pd.read_csv(calcloc + '02_china900_codes.csv', sep=';')
    df = df.sort_values(['Next_Trading_Day', 'ISIN'], ascending=[True, True]).reset_index(drop=True)
    df['dt2']=pd.to_datetime(df['Next_Trading_Day'], format='%Y-%m-%d', dayfirst=True)

    dfds = pd.read_excel(histloc + '/universe/DSCODE_map.xlsx')

    dto = np.datetime64('2050-01-01')
    dfds['dtto'] = dfds['dtto'].fillna(dto)

    df = df.iloc[:,:]

    infocode_list = []
    dfids = df[['sedol','ISIN']].drop_duplicates().reset_index(drop=True)
    print(len(dfids))
    for index, row in dfids.iterrows():

        if np.isnan(q.get_infocode(str(row.sedol))):
            try:
                infocode = q.get_infocode(dfids.loc[index,'ISIN'])
            except:
                infocode = ''
        else:
            infocode = q.get_infocode(str(row.sedol))

        infocode_list.append(infocode)

        if not (index+1) % 300:
            print(index+1)

    dfids['infocode'] = infocode_list
    df = pd.merge(df, dfids, how='left', on=['sedol','ISIN']).drop_duplicates()
    # -----------

    # get infocode from DSCODE_map 
    for index, row in dfds.iterrows():
        df.loc[df[(df.Internal_Number==row.symbol) & (df.dt2>=row.dtfrom) & (df.dt2<=row.dtto)].index, 'infocode'] = row.infocode
    # -----------

    del df['dt2']

    df.to_csv(calcloc + '03_china900_codes.csv', sep=';', index=False)

    print('done step2')

In [32]:
do_step1()
do_step2()

1543
300
600
900
1200
1500
done step1
1543
300
600
900
1200
1500
done step2


In [44]:
#download

#data download example

dfin = pd.read_csv(calcloc + '03_china900_codes.csv', sep=';', dtype={'sedol': object})
#dfin.groupby('Date').count()['symbol'].plot()
dfin['Date'] = pd.to_datetime(dfin['Next_Trading_Day'], format='%Y.%m.%d', dayfirst=True)
dfin['cutoff'] = dfin['Date']
dfin['cutoff'] = dfin['cutoff'].map(lambda x: x-dt.timedelta(days=(x.day)))

dfin = dfin[dfin['Date'].map(lambda x: x.month==6 or x.month==12)]

time1=time.time()

dfin = dfin.iloc[:,:]

dfin['sedol_map'] = dfin['sedol'].map(lambda x: x[:6])

#dfin = dfin[['Date','cutoff','sedol','sedol_map']][:] #### reduce for testing

def get_sqlstr(sedlst):
    sedstr = ''
    for s in sedlst:
        sedstr = sedstr + s[:6] + ","
    return sedstr[:-1].replace("'","")

#'3995', #Total Shareholders’ Equity
#'3255', #Total Debt
#'2999', #Total Assets
#'2501', #Property, Plant And Equipment - Net
#'2101', #Inventories - Total
#'2051', #Receivables (Net)
#'4860', #NET CASH FLOW from OPERATING ACTIVITIES (12 Month trailing)
#'NI', #1551-1505: Net Income = Net Income Before Extraordinary Items/Preferred Dividends - Discontinued Operations

items_agg = ['1551','1505']
items_last = ['3995','3255','2999','2501','2101','2051','4860']

df2 = pd.DataFrame()

print(len(dfin))
ct=0
print(len(dfin['cutoff'].drop_duplicates()))
for d in sorted(dfin['cutoff'].drop_duplicates().reset_index(drop=True)):

    sedlst =list(dfin[dfin.cutoff==d]['sedol'])
    sedstr = get_sqlstr(sedlst)
    
    df1 = dfin[dfin.cutoff==d].reset_index(drop=True) #[['Date','cutoff','sedol','sedol_map']].reset_index(drop=True)
    
    for m in range(0,5):
        #print(m)
        for i_a in items_agg:
            #print(i_a)
            dat = str(myf.add_months(d,-m*12))[:]
            #print(dat)
            dffund = q.get_prod_wswspit(i_a, sedstr, dat, 'N','wspit', -12)[['sedol','value']]
            dffund['value'] = dffund['value']
            dffund = dffund.rename(columns={'sedol':'sedol_map','value':i_a + '-'+str(m)})
            if len(dffund)==0:
                print(dat, i_a + '-'+str(m), ' - zero records')
            
            df1 = pd.merge(df1, dffund, on='sedol_map', how='left')
    
    dat = str(d)[:10]
    for i_lst in items_last:
        
        if (i_lst=='4860'):
            dffund = q.get_prod_wswspit(i_lst, sedstr, dat, 'N', 'wspit', -12)[['sedol','value']]
            dffund['value'] = dffund['value']
        else:
            dffund = q.get_prod_wswspit(i_lst, sedstr, dat, 'Y', 'wspit', -12)[['sedol','value']]
            dffund['value'] = dffund['value']
            
        dffund = dffund.rename(columns={'sedol':'sedol_map','value':i_lst})
        
        if len(dffund)==0:
            print(dat, i_lst, ' - zero records')
            
        df1 = pd.merge(df1, dffund, on='sedol_map', how='left')
            
    if len(df2)==0:
        df2 = df1
    else:
        df2 = pd.concat([df2, df1], axis=0)
        
    print(ct)
    ct=ct+1

time2=time.time()
print('fundamental download - time elapsed:', (time2-time1)/60, 'minutes')

del df2['sedol_map']

for m in range(0,5):
    df2['1505-'+str(m)] = df2['1505-'+str(m)].fillna(0)
    df2['NI-'+str(m)] = df2['1551-'+str(m)] - df2['1505-'+str(m)]
    del df2['1551-'+str(m)]
    del df2['1505-'+str(m)]

def nistdev(series): 
    for i in range(len(series)):
        if np.isnan(series[i]):
            series[i] = 0
    ni_h=np.array(list([series[0], series[1], series[2], series[3], series[4]]))
    if len(ni_h[ni_h!=0])>=3:
        return np.std(ni_h[ni_h!=0], ddof=1)
    else:
        return np.nan

df2['NIstdev'] = df2[['NI-0', 'NI-1', 'NI-2', 'NI-3', 'NI-4']].apply(nistdev, axis=1)


df2.drop_duplicates().to_csv(calcloc + '04_china900_fundamentals.csv' ,sep=';',index=False)

df2

11700
13
2007-11-30 1505-3  - zero records
0
2008-05-31 1505-3  - zero records
2007-05-31 1505-4  - zero records
1
2007-11-30 1505-4  - zero records
2
2008-05-31 1505-4  - zero records
3
4
5
6
7
8
9
10
11
12
fundamental download - time elapsed: 7.358169078826904 minutes


,Next_Trading_Day,Index_Name,Index_Currency,ISIN,SEDOL,RIC,Instrument_Name,Country,Currency,Exchange,...,2501,2101,2051,4860,NI-0,NI-1,NI-2,NI-3,NI-4,NIstdev
0,2010-12-20,STXCNA900 EUR P,EUR,CNE000000024,NaN,000501.SZ,NaN,CN,CNY,NaN,...,2144.877000,474.745000,314.063000,1358.828000,284.485000,252.160000,138.949000,34.420000,20.273469,121.197151
1,2010-12-20,STXCNA900 EUR P,EUR,CNE000000040,NaN,000001.SZ,NaN,CN,CNY,NaN,...,2403.605000,NaN,NaN,4626.067000,5030.729000,614.035000,2649.903000,1463.386000,286.493000,1920.433227
2,2010-12-20,STXCNA900 EUR P,EUR,CNE0000000C8,NaN,000659.SZ,NaN,CN,CNY,NaN,...,3399.495000,455.797000,875.332000,442.703000,124.019000,115.156000,54.292000,124.475000,75.771000,31.913958
3,2010-12-20,STXCNA900 EUR P,EUR,CNE0000000J3,NaN,000677.SZ,NaN,CN,CNY,NaN,...,4002.131000,1014.836000,426.992000,187.785000,58.000000,-178.543000,242.793000,189.917000,67.374000,162.778601
4,2010-12-20,STXCNA900 EUR P,EUR,CNE0000000M7,NaN,000637.SZ,NaN,CN,CNY,NaN,...,204.814000,82.647000,21.622000,89.909000,99.362000,52.327000,84.975000,179.238000,25.406000,58.422701
5,2010-12-20,STXCNA900 EUR P,EUR,CNE0000000S4,NaN,000811.SZ,NaN,CN,CNY,NaN,...,371.038000,348.214000,297.289000,111.825000,124.508000,52.695000,44.906000,66.187000,38.384000,34.661361
6,2010-12-20,STXCNA900 EUR P,EUR,CNE0000000T2,NaN,000002.SZ,NaN,CN,CNY,NaN,...,2256.396655,123552.776633,24866.376628,-6508.577363,5644.149396,4714.220466,5203.890497,2761.400563,1933.544000,1616.619044
7,2010-12-20,STXCNA900 EUR P,EUR,CNE000000107,NaN,001696.SZ,NaN,CN,CNY,NaN,...,465.321000,154.954000,1319.315000,70.743000,354.226000,350.812000,279.912000,189.974000,111.969000,105.242939
8,2010-12-20,STXCNA900 EUR P,EUR,CNE000000115,NaN,000511.SZ,NaN,CN,CNY,NaN,...,151.997000,2853.153000,49.782000,-1151.088000,77.411000,87.235000,141.280000,168.737000,41.341000,51.214437
9,2010-12-20,STXCNA900 EUR P,EUR,CNE000000123,NaN,000046.SZ,NaN,CN,CNY,NaN,...,109.162000,15543.302000,380.638000,-4225.832000,367.655000,218.029000,541.265000,701.838000,95.623000,243.271562


In [134]:
# adtv

cur= 'CNY'

dfin = pd.read_csv(calcloc + '04_china900_fundamentals.csv', sep=';', dtype={'sedol': object})
#dfin.groupby('Date').count()['symbol'].plot()
dfin['Date'] = pd.to_datetime(dfin['Date'], format='%Y.%m.%d', dayfirst=True)
dfin['cutoff'] = pd.to_datetime(dfin['cutoff'], format='%Y.%m.%d', dayfirst=True)

f='SXCNA9P' #'SX60JP'
url='http://www.stoxx.com/download/historical_data/h_'+f.lower()+'.txt'
#print(url)
dfweb = myf.get_h([f], histloc, usedates=True)
dfweb['Date']=pd.to_datetime(dfweb['Date'], format='%d.%m.%Y', dayfirst=True)
dfweb = dfweb[-dfweb[f].isnull()].sort_values('Date', ascending=True).reset_index(drop=True)

time1=time.time()

wst=[]
wslist=['adtv', # 0
        'ctdays', # 1
        'sedol', # 2
        'Date'] # 3

ct=0
dfin = dfin.iloc[:,:]
print(len(dfin))

for sed in (dfin['sedol']).drop_duplicates().reset_index(drop=True):
    
    #print(sed)
    
    dfsed = dfin[dfin.sedol==sed].reset_index(drop=True)
    wsdata = [[0 for x in range(len(wslist))] for x in range(len(dfsed))]
    #cur = dfsed.loc[0, 'curr']
    
    for index, row in dfsed.iterrows():
        
        #d=pd.to_datetime(row.Date, format='%Y-%m-%d', dayfirst=True)
        #d=dt.datetime.strptime(row.Date, '%Y-%m-%d')
        d = row.cutoff
        d1=d + dt.timedelta(days=1) #first day of the month - needed for adtv calc

        
        #adtv import and calculation
        if not np.isnan(row.infocode):
            adtv_= q.get_adtv_prim_plus(row.sedol, d, myf.add_months(d,-13), cur, int(row.infocode))
        else:
            adtv_= q.get_adtv_prim_plus(row.sedol, d, myf.add_months(d,-13), cur)
        dfctdays = dfweb[(dfweb['Date']<=d) & (dfweb['Date']>=myf.add_months(d1,-3))].sort_values('Date', ascending=False).reset_index(drop=True)
        
        
        if len(adtv_)==0:
            wsdata[index][0] = 0
        else: 
            adtv_=adtv_[-adtv_.close_.isnull()].sort_values('marketdate', ascending=False).reset_index(drop=True)
            
            if len(dfctdays)>0:
                dfadtv = pd.merge(dfctdays, adtv_, how='left', left_on='Date', right_on='marketdate').iloc[:,[0,3,4]] #left join to get sxjp600 days, even if zero value
                wsdata[index][0] = (dfadtv.loc[:,'close_'] * dfadtv.loc[:,'volume']).sum() / len(dfctdays)
                wsdata[index][1] = len(dfctdays)
            else:
                wsdata[index][0] = 0
                wsdata[index][1] = 0

        wsdata[index][2] = sed
        wsdata[index][3] = row.Date
        
        if ct==0:
            wsdataall = pd.DataFrame(wsdata, columns=wslist)
        else:
            wsdataall = pd.concat([wsdataall, pd.DataFrame(wsdata, columns=wslist)], axis=0)
        
        if not (ct+1) % 500:
            print(' -- ', ct+1, ' -- ')
        
        ct=ct+1

del wsdataall['ctdays']

wsdataall['sedol'] = wsdataall['sedol'].map(lambda x: str(x))
wsdataall['Date'] = wsdataall['Date'].map(lambda x: str(x)[:10])
dfin['Date'] = dfin['Date'].map(lambda x: str(x)[:10])

dfin = pd.merge(dfin, wsdataall, how='left', left_on=['Date','sedol'], right_on=['Date','sedol']).sort_values(['Date','ISIN'], ascending=[True, True]).drop_duplicates()        

time2=time.time()
print('time elapsed:', (time2-time1)/60, 'minutes')

dfin.drop_duplicates().to_csv(calcloc + '05_china900_alldata.csv' ,sep=';',index=False)
print('done')

http://www.stoxx.com/download/historical_data/h_sxcna9p.txt
11700
 --  500  -- 
 --  1000  -- 
 --  1500  -- 
 --  2000  -- 
 --  2500  -- 
 --  3000  -- 
 --  3500  -- 
 --  4000  -- 
 --  4500  -- 
 --  5000  -- 
 --  5500  -- 
 --  6000  -- 
 --  6500  -- 
 --  7000  -- 
 --  7500  -- 
 --  8000  -- 
 --  8500  -- 
 --  9000  -- 
 --  9500  -- 
 --  10000  -- 
 --  10500  -- 
 --  11000  -- 
 --  11500  -- 


ValueError: time data '0' does match format specified

In [72]:
#selection
import scipy.stats as stats

#delete REITs?
#icb_reit = ['8671','8672','8673','8674','8675','8676','8677','8737','8733',8671,8672,8673,8674,8675,8676,8677,8737,8733,'REA']
#dfin = dfin[-dfin.icb_sub.isin(list(icb_reit))]

dfjp = pd.read_csv(calcloc + '05_china900_alldata.csv' ,sep=';')
dfjp['Date'] = dfjp['Date'].map(lambda x: pd.to_datetime(str(x)[:10], format='%Y-%m-%d', dayfirst=True))
dfjp['cutoff'] = dfjp['cutoff'].map(lambda x: pd.to_datetime(str(x)[:10], format='%Y-%m-%d', dayfirst=True))

dfjp = dfjp.rename(columns={'Mcap_Units_Index_Currency':'ffmcap_cny'})

#'3995', #Total Shareholders’ Equity
#'3255', #Total Debt
#'2999', #Total Assets
#'2501', #Property, Plant And Equipment - Net
#'2101', #Inventories - Total
#'2051', #Receivables (Net)
#'4860', #NET CASH FLOW from OPERATING ACTIVITIES (12 Month trailing)
#'NI', #1551-1505: Net Income = Net Income Before Extraordinary Items/Preferred Dividends - Discontinued Operations

dfjp[['2501','2101','2051']].fillna(0, inplace=True)
dfjp['ROE'] = dfjp['NI-0'] / dfjp['3995']
dfjp['FinHe'] = dfjp['3255'] / (dfjp['3995'] + dfjp['3255'])

dfjp['CFGenInd'] = dfjp.loc[dfjp[dfjp['ICB_ind']!=8000].index, '4860'] / (dfjp['2501'] + dfjp['2101'] + dfjp['2051'])
dfjp['CFGenFin'] = dfjp.loc[dfjp[dfjp['ICB_ind']==8000].index, '4860'] / dfjp['2999']
dfjp['Stability'] = dfjp['NIstdev'] / dfjp['3995']

dfjp['ROE_missing'] = 'ok'
dfjp.loc[dfjp[(dfjp['3995'].isnull()) | (dfjp['3995']<=0) | (dfjp['NI-0'].isnull())].index,'ROE_missing'] = 'missing'

dfjp['FinHe_missing'] = 'ok'
dfjp.loc[dfjp[(dfjp['3995'].isnull()) | (dfjp['3995']<=0) | (dfjp['3255'].isnull())].index,'FinHe_missing'] = 'missing'

dfjp['CF_ind_missing'] = 'ok'
dfjp.loc[dfjp[(dfjp['CFGenInd'].isnull()) | (dfjp['4860'].isnull()) | (dfjp['4860']<=0) | 
              ((dfjp['2501'] + dfjp['2101'] + dfjp['2051'])==0)].index,'CF_ind_missing'] = 'missing'
dfjp['CF_fin_missing'] = 'ok'
dfjp.loc[dfjp[(dfjp['CFGenFin'].isnull()) | (dfjp['4860'].isnull()) 
              | (dfjp['4860']<=0) | (dfjp['2999']<=0)].index,'CF_fin_missing'] = 'missing'
               
#calculate ranking for scoring - using mcap as a second ranking criterion
for d in dfjp.sort('Date', ascending=True)['Date'].drop_duplicates():

    #adtv
    dfidx1 = dfjp[dfjp.Date==d].sort_values(['adtv','ffmcap_cny'], ascending=[False, False])[['adtv','ffmcap_cny']].index
    rk1 = np.arange(len(dfjp[dfjp.Date==d])-1, -1, -1)
    dfjp.loc[dfidx1, 'rkadtv'] = rk1
    dfjp.loc[dfidx1, 'rkadtv'] = dfjp.loc[dfidx1, 'rkadtv'].map(lambda x: stats.percentileofscore(rk1, x, kind='strict')/100.)
    dfjp.loc[dfidx1, 'rkadtv'] = dfjp.loc[dfidx1, 'rkadtv'] / max(dfjp.loc[dfidx1, 'rkadtv'])

    #ROE
    dfidx1 = dfjp[(dfjp.Date==d) & (dfjp.ROE_missing!='missing')].sort_values(['ROE','ffmcap_cny'], ascending=[False, False])[['ROE','ffmcap_cny']].index
    rk1 = np.arange(len(dfjp[(dfjp.Date==d) & (dfjp.ROE_missing!='missing')])-1, -1, -1)
    dfjp.loc[dfidx1, 'rkROE'] = rk1
    dfjp.loc[dfidx1, 'rkROE'] = dfjp.loc[dfidx1, 'rkROE'].map(lambda x: stats.percentileofscore(rk1, x, kind='strict')/100.)
    dfjp.loc[dfidx1, 'rkROE'] = dfjp.loc[dfidx1, 'rkROE'] / max(dfjp.loc[dfidx1, 'rkROE'])
    
    #FinHealth
    dfidx1 = dfjp[(dfjp.Date==d) & (dfjp.FinHe_missing!='missing')].sort_values(['FinHe','ffmcap_cny'], ascending=[True, False])[['FinHe','ffmcap_cny']].index
    rk1 = np.arange(len(dfjp[(dfjp.Date==d) & (dfjp.FinHe_missing!='missing')])-1, -1, -1)
    dfjp.loc[dfidx1, 'rkFinHe'] = rk1
    dfjp.loc[dfidx1, 'rkFinHe'] = dfjp.loc[dfidx1, 'rkFinHe'].map(lambda x: stats.percentileofscore(rk1, x, kind='strict')/100.)
    dfjp.loc[dfidx1, 'rkFinHe'] = dfjp.loc[dfidx1, 'rkFinHe'] / max(dfjp.loc[dfidx1, 'rkFinHe'])
    
    #CF
    dfidx1 = dfjp[(dfjp.Date==d) & (dfjp.CF_ind_missing!='missing')].sort_values(['CFGenInd','ffmcap_cny'], ascending=[False, False])[['CFGenInd','ffmcap_cny']].index
    rk1 = np.arange(len(dfjp[(dfjp.Date==d) & (dfjp.CF_ind_missing!='missing')])-1, -1, -1)
    dfjp.loc[dfidx1, 'rkCFind'] = rk1
    dfjp.loc[dfidx1, 'rkCFind'] = dfjp.loc[dfidx1, 'rkCFind'].map(lambda x: stats.percentileofscore(rk1, x, kind='strict')/100.)
    dfjp.loc[dfidx1, 'rkCFind'] = dfjp.loc[dfidx1, 'rkCFind'] / max(dfjp.loc[dfidx1, 'rkCFind']) 
    rkCFGenInd = dfjp['rkCFind']
    
    dfidx1 = dfjp[(dfjp.Date==d) & (dfjp.CF_fin_missing!='missing')].sort_values(['CFGenFin','ffmcap_cny'], ascending=[False, False])[['CFGenFin','ffmcap_cny']].index
    rk1 = np.arange(len(dfjp[(dfjp.Date==d) & (dfjp.CF_fin_missing!='missing')])-1, -1, -1)
    dfjp.loc[dfidx1, 'rkCFfin'] = rk1
    dfjp.loc[dfidx1, 'rkCFfin'] = dfjp.loc[dfidx1, 'rkCFfin'].map(lambda x: stats.percentileofscore(rk1, x, kind='strict')/100.)
    dfjp.loc[dfidx1, 'rkCFfin'] = dfjp.loc[dfidx1, 'rkCFfin'] / max(dfjp.loc[dfidx1, 'rkCFfin'])
    
    rkCFGenInd = dfjp['rkCFind']
    rkCFGenInd[isnan(rkCFGenInd)]=0
    rkCFGenFin = dfjp['rkCFfin']
    rkCFGenFin[isnan(rkCFGenFin)]=0
    rkCFGen = rkCFGenInd + rkCFGenFin
    dfjp['rkCFGen'] = rkCFGen

    #Business Stability (use ROE_missing condition)
    dfidx1 = dfjp[(dfjp.Date==d) & (dfjp.ROE_missing!='missing')].sort_values(['Stability','ffmcap_cny'], ascending=[True, False])[['FinHe','ffmcap_cny']].index
    rk1 = np.arange(len(dfjp[(dfjp.Date==d) & (dfjp.ROE_missing!='missing')])-1, -1, -1)
    dfjp.loc[dfidx1, 'rkStab'] = rk1
    dfjp.loc[dfidx1, 'rkStab'] = dfjp.loc[dfidx1, 'rkStab'].map(lambda x: stats.percentileofscore(rk1, x, kind='strict')/100.)
    dfjp.loc[dfidx1, 'rkStab'] = dfjp.loc[dfidx1, 'rkStab'] / max(dfjp.loc[dfidx1, 'rkStab'])

    
#treatment of missing values - #################tbc
dfjp.loc[(dfjp[dfjp.ROE_missing=='missing']).index, ['rkROE','rkStab']] = 0
dfjp.loc[(dfjp[dfjp.FinHe_missing=='missing']).index, 'rkFinHe'] = 0
dfjp.loc[(dfjp[(dfjp.CF_ind_missing=='missing') & (dfjp.CF_fin_missing=='missing')]).index, 'rkCFGen'] = 0

#screening
dfjp['ROE_screen'] = ''
idxout = dfjp[dfjp.rkROE<0.5].index
dfjp.loc[idxout, 'ROE_screen'] = 'out'

dfjp['other3_screen'] = ''
idxout = dfjp[(dfjp.rkStab<0.05) | (dfjp.rkFinHe<0.05) | (dfjp.rkCFGen<0.05)].index
dfjp.loc[idxout, 'other3_screen'] = 'out'

#add stocks connect eligibility
dfjp.loc[dfjp[(dfjp['ROE_screen']=='out') | (dfjp['other3_screen']=='out')].index,'eligible'] = 'FAIL'
dfjp.loc[(dfjp[dfjp.eligible!='FAIL']).index, 'eligible'] = 'PASS'

#composite rank
dfjp['rKComp'] = 0.4*dfjp['rkROE'] + 0.2*(dfjp['rkFinHe'] + dfjp['rkCFGen'] + dfjp['rkStab'])

#selection
dfjp['old'] = False
#dfjp['rkFinal'] = 0
dfjp['new'] = False

ct=0
for d in dfjp.sort_values('Date', ascending=True)['Date'].drop_duplicates():
    ct=ct+1
    if ct>1:
        #mark the current components
        dfnew = dfjp[(dfjp.Date==d)][['infocode']] #take all current candidates
        dfnew = pd.merge(dfnew, dfold, how='left', on='infocode') #link old (column new)
        dfnew['new'] = dfnew['new'].fillna(False)
        listold = dfnew['new'].tolist()
        dfjp.loc[dfjp[dfjp.Date==d].index,'old'] = listold
        #a= bcd
    
    idxrk = dfjp[(dfjp.Date==d) & (dfjp.eligible=='PASS') & ((dfjp.old==True) | ((dfjp.old==False) & (dfjp.rkadtv>=0.2)))].index
    rk150 = dfjp.loc[idxrk, 'rkComp'].rank(method='first', ascending=False)
    dfjp.loc[idxrk, 'rkFinal'] = rk150
    dfjp.loc[idxrk, 'new'] = rk150<=150
    
    dfold=dfjp[(dfjp.Date==d) & (dfjp.new==True)][['infocode','new']]
    
#weighting
dfjp = dfjp.sort_values(['Date','ffmcap_cny'], ascending=[True, False])
dfjp.reset_index(inplace=True,drop=True)

dfjp.loc[dfjp[dfjp.new==True].index,'wgt'] = 0
dfjp.loc[dfjp[dfjp.new==True].index,'cap'] = 0.02

sumcap = dfjp[dfjp.new==True].groupby('Date').sum()['ffmcap_cny']

for i in range(len(sumcap)):
    
    wgt = dfjp.loc[dfjp[(dfjp.Date==sumcap.index[i]) & (dfjp.new==True)].index, 'ffmcap_cny'] / sumcap[i]
    dfjp.loc[wgt.index,'wgt'] = wgt
    
    dfftrs = myf.calccapfacs(dfjp.loc[wgt.index,['wgt','cap']])
    
    dfftrs.index = wgt.index #reindex the capfactor df so corresponds to the target df indices
    #dfjp.loc[dfftrs.index,'capfactor'] = dfftrs['capfactor']
    dfjp.loc[dfftrs.index,'cappedwgt'] = dfftrs['cappedwgt']

dfjp.sort_values(['Date','new'], ascending=[True,False]).to_csv(calcloc + '06_china900_selection.csv' ,sep=';',index=False)

print('weighting done')

#IF files
dfjp['CUSIP'] = np.nan
idxname = ['ISMCAP', 'ISMCAN', 'ISMCAG']

for z in range(len(idxname)):
    print(idxname[z])
    for d in dfjp.sort_values('Date', ascending=True)['Date'].drop_duplicates():
        dfif = dfjp[(dfjp.Date==d) & (dfjp.new==True)][['SEDOL','CUSIP','ISIN','cappedwgt']]
        dfif.columns = ['SEDOL<string>','CUSIP<string>','ISIN<string>','Weight']
        dfif.to_csv(histloc + '02_IF/' + idxname[z] + '_' + str(d)[:10].replace('-','') + '.csv', float_format='%.9f',index=False)

print('done')

C:\Users\iv822\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\frame.py:2762: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)
C:\Users\iv822\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:45: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
C:\Users\iv822\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\iv822\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

S

weighting done
ISMCAP
ISMCAN
ISMCAG
done
